In [1]:
import subprocess
from subprocess import Popen, PIPE # Import Popen and PIPE
import json
import pandas as pd
from pathlib import Path
import glob
import os
import re # For parsing output
from tqdm import tqdm

# --- Configuration ---
# URL of the Dementia Diaries SoundCloud user tracks page
SOUNDCLOUD_URL = "https://soundcloud.com/dementia-diaries/tracks"

# Directory to save the downloaded audio files and metadata
# Adjust the relative path if necessary, e.g., go up one level from 'ml' then into 'data'
output_base_dir = Path('../data/raw/dementia_diaries')

# Output WAV format settings
TARGET_SAMPLE_RATE = 16000
AUDIO_FORMAT = "wav"

# File to track downloaded files (for resuming)
ARCHIVE_FILE = output_base_dir / "download_archive.txt"

# --- Ensure output directory exists ---
output_base_dir.mkdir(parents=True, exist_ok=True)

print(f"SoundCloud URL: {SOUNDCLOUD_URL}")
print(f"Output Directory: {output_base_dir}")
print(f"Target Sample Rate: {TARGET_SAMPLE_RATE} Hz")
print(f"Target Format: {AUDIO_FORMAT}")

SoundCloud URL: https://soundcloud.com/dementia-diaries/tracks
Output Directory: ..\data\raw\dementia_diaries
Target Sample Rate: 16000 Hz
Target Format: wav


In [2]:
# Cell 2: Construct and Run yt-dlp Command with Selective Real-time Output & Progress

# Re-define config here just in case cell is run independently,
# though ideally it's set in Cell 1
SOUNDCLOUD_URL = "https://soundcloud.com/dementia-diaries/tracks"
output_base_dir = Path('../data/raw/dementia_diaries')
TARGET_SAMPLE_RATE = 16000
AUDIO_FORMAT = "wav"
ARCHIVE_FILE = output_base_dir / "download_archive.txt"
output_base_dir.mkdir(parents=True, exist_ok=True) # Ensure dir exists

# --- yt-dlp Command Construction ---

# Template for output files: Create a subdirectory named 'audio'
# Save files as: audio/TITLE_trackID.wav
output_template = str(output_base_dir / "audio/%(title)s_%(id)s.%(ext)s")
TOTAL_TRACKS = 3628 # Approximate total from SoundCloud page - update if necessary

command = [
    "yt-dlp",
    # Add '--no-progress' to potentially reduce fragment download lines,
    # but we'll filter manually anyway. Optional.
    # "--no-progress",
    "--ignore-errors", # Continue if some tracks fail
    "--continue",      # Continue if previous download was interrupted
    "--download-archive", str(ARCHIVE_FILE), # Track completed downloads
    "--extract-audio", # Download only audio
    "--audio-format", AUDIO_FORMAT, # Convert to WAV
    "--audio-quality", "0", # Best quality for conversion source
    # Use postprocessor args to resample to the target rate *after* conversion
    "--postprocessor-args", f"ffmpeg:-ar {TARGET_SAMPLE_RATE}", # Correct way to pass ffmpeg args
    "--write-info-json", # Save metadata to a .info.json file
    # Sleep a bit between downloads to be polite to SoundCloud servers
    "--sleep-interval", "1", # Reduced sleep interval slightly
    "--max-sleep-interval", "3", # Reduced sleep interval slightly
    # Define the output filename structure
    "-o", output_template,
    # The URL to download from
    SOUNDCLOUD_URL
]

print("--- Running yt-dlp Command ---")
# Using list for command printing avoids quoting issues on different OS
print(command)
print(f"\nAttempting to download up to ~{TOTAL_TRACKS} tracks (will skip already downloaded)...")
print(f"Output Dir: {output_base_dir / 'audio'}")
print(f"Archive File: {ARCHIVE_FILE}")
print("(Only key status messages like '100%' or 'already downloaded' will be printed below)")

# --- Execute with Popen for real-time output and progress ---
process = None
pbar = None # Initialize pbar outside try block
try:
    # Use Popen to run the command in the background
    # Use text=True (or encoding) and errors='replace' for safer text handling
    process = Popen(command, stdout=PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8', errors='replace', bufsize=1)

    # Regex to find lines indicating track number, e.g., "[soundcloud] 12345678: Downloading track 5 of 3628"
    track_progress_regex = re.compile(r"Downloading track (\d+)(?: of \d+)?")
    # Keywords to identify lines we *do* want to print
    keywords_to_print = ["[download] 100%", "has already been downloaded", "Downloading track", "[ExtractAudio]", "[ffmpeg]"]

    # Initialize tqdm progress bar
    pbar = tqdm(total=TOTAL_TRACKS, unit="track", desc="Downloading Diaries")

    # Check how many already downloaded from archive file for initial progress
    initial_done = 0
    if ARCHIVE_FILE.exists():
         try:
              with open(ARCHIVE_FILE, 'r') as af:
                   initial_done = sum(1 for _ in af)
              pbar.update(initial_done)
              print(f"Resuming download, {initial_done} tracks already in archive.")
         except Exception as e:
              print(f"Could not read archive file {ARCHIVE_FILE}: {e}")

    current_track = initial_done
    last_reported_track = initial_done

    # Read stdout line by line
    if process.stdout:
        for line in iter(process.stdout.readline, ''):
            # Print the line ONLY if it contains one of the keywords
            if any(keyword in line for keyword in keywords_to_print):
                print(line, end='')

            # --- Progress Bar Update Logic (independent of printing) ---
            match = track_progress_regex.search(line)
            if match:
                # Extract track number if possible
                current_track_num_in_line = int(match.group(1))
                # Sometimes yt-dlp repeats lines for same track, only update if number increases
                if current_track_num_in_line > last_reported_track:
                     update_amount = current_track_num_in_line - last_reported_track
                     pbar.update(update_amount)
                     last_reported_track = current_track_num_in_line

            elif "[download] 100%" in line:
                 # Fallback: If a download completes but track number wasn't parsed,
                 # cautiously advance progress by 1 if we haven't reached the total
                 if last_reported_track < pbar.total:
                      # Check if the line containing 100% also contained the track number update
                      if not match: # Only update if the primary regex didn't catch it on this line
                        maybe_next_track = last_reported_track + 1
                        if maybe_next_track <= pbar.total: # Ensure we don't exceed total
                            pbar.update(1)
                            last_reported_track = maybe_next_track
            # --- End Progress Bar Update Logic ---

        process.stdout.close()

    # Wait for the process to finish and get return code
    return_code = process.wait()
    pbar.close() # Close progress bar

    if return_code == 0:
        print("\n--- yt-dlp process completed successfully ---")
        # Check archive file for final count
        final_count = 0
        try:
            with open(ARCHIVE_FILE, 'r') as af:
                final_count = sum(1 for _ in af)
            print(f"Archive file indicates {final_count} tracks marked as downloaded.")
            # Ensure progress bar shows accurate final count
            if final_count != pbar.n:
                 pbar.reset(total=TOTAL_TRACKS) # Reset if necessary
                 pbar.update(final_count)
                 pbar.refresh()

        except FileNotFoundError:
             print("Archive file not found after run.")
        except Exception as e:
             print(f"Error reading final archive count: {e}")

    else:
        print(f"\n--- yt-dlp process finished with errors (Return Code: {return_code}) ---")
        if process.stderr:
            print("--- STDERR ---")
            # Read and print stderr only if there was an error
            stderr_output = ""
            try:
                stderr_output = process.stderr.read()
                print(stderr_output)
            except Exception as e:
                print(f"Error reading stderr: {e}")
            finally:
                 if not process.stderr.closed:
                      process.stderr.close()
        print("Download may be incomplete. Check logs and the archive file.")


except FileNotFoundError:
    if pbar: pbar.close()
    print("\n--- Error: 'yt-dlp' command not found ---")
    print("Please ensure yt-dlp is installed and in your system's PATH.")
    print("Install using: pip install yt-dlp")
except KeyboardInterrupt:
    if pbar: pbar.close()
    print("\n--- Download Interrupted by User (Ctrl+C) ---")
    if process:
        print("Attempting to terminate yt-dlp...")
        process.terminate() # Try to terminate gracefully
        try:
            process.wait(timeout=5) # Wait a bit
        except subprocess.TimeoutExpired:
            print("Process did not terminate gracefully, killing.")
            process.kill() # Force kill if needed
        print("yt-dlp process terminated.")
    print("Completed downloads are saved in the archive file. Run the script again to resume.")
except Exception as e:
    if pbar: pbar.close()
    print(f"\n--- An unexpected error occurred ---")
    print(e)
    if process and process.stderr and not process.stderr.closed:
        try:
             print("--- Last STDERR ---")
             print(process.stderr.read())
        except Exception as serr:
             print(f"Error reading stderr: {serr}")

finally:
    # Ensure streams are closed if process exists
     if process:
          if process.stdout and not process.stdout.closed:
               process.stdout.close()
          if process.stderr and not process.stderr.closed:
               process.stderr.close()

--- Running yt-dlp Command ---
['yt-dlp', '--ignore-errors', '--continue', '--download-archive', '..\\data\\raw\\dementia_diaries\\download_archive.txt', '--extract-audio', '--audio-format', 'wav', '--audio-quality', '0', '--postprocessor-args', 'ffmpeg:-ar 16000', '--write-info-json', '--sleep-interval', '1', '--max-sleep-interval', '3', '-o', '..\\data\\raw\\dementia_diaries\\audio\\%(title)s_%(id)s.%(ext)s', 'https://soundcloud.com/dementia-diaries/tracks']

Attempting to download up to ~3628 tracks (will skip already downloaded)...
Output Dir: ..\data\raw\dementia_diaries\audio
Archive File: ..\data\raw\dementia_diaries\download_archive.txt
(Only key status messages like '100%' or 'already downloaded' will be printed below)


Resuming download, 346 tracks already in archive.
[soundcloud:user] 140064103: Downloading track page 1
[soundcloud:user] 140064103: Downloading track page 2
[soundcloud:user] 140064103: Downloading track page 3
[soundcloud:user] 140064103: Downloading track page 4
[soundcloud:user] 140064103: Downloading track page 5
[soundcloud:user] 140064103: Downloading track page 6
[soundcloud:user] 140064103: Downloading track page 7
[soundcloud:user] 140064103: Downloading track page 8
[soundcloud:user] 140064103: Downloading track page 9
[soundcloud:user] 140064103: Downloading track page 10
[soundcloud:user] 140064103: Downloading track page 11
[soundcloud:user] 140064103: Downloading track page 12
[soundcloud:user] 140064103: Downloading track page 13
[soundcloud:user] 140064103: Downloading track page 14
[soundcloud:user] 140064103: Downloading track page 15
[soundcloud:user] 140064103: Downloading track page 16
[soundcloud:user] 140064103: Downloading track page 17
[soundcloud:user] 140064

[download] 100% of  429.27KiB in 00:00:00 at 803.52KiB/s               
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\Wise words from Howard as he muses on the weather._1084512442.wav
[download] 100% of  603.11KiB in 00:00:00 at 833.26KiB/s                 
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\Joy is gobsmacked,�and deservedly so X_1080174700.wav
[download] 100% of  744.63KiB in 00:00:00 at 1004.09KiB/s                
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\James wonders about �reminiscence rooms�_1080174424.wav


[download] 100% of  599.31KiB in 00:00:00 at 859.12KiB/s               
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\James has good advice on the storage of important personal medications in hospital._1080174070.wav


[download] 100% of  633.98KiB in 00:00:00 at 994.78KiB/s                 
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\James feels that hospitals should make it easier to give anonymous feedback._1080173740.wav


...

[download] 100% of    1.71MiB in 00:00:01 at 1.24MiB/s                 
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\Keith wonders if, like him, others sometimes feel redundant._645169143.wav


[download] 100% of  663.82KiB in 00:00:00 at 936.52KiB/s               
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\Julie talks about the fatigue she is experiencing._645168639.wav


[download] 100% of  939.35KiB in 00:00:00 at 1.14MiB/s                   
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\Rae wonders whether a friend is making the right decisions._641762643.wav


[download] 100% of    3.12MiB in 00:00:02 at 1.36MiB/s                 
[ExtractAudio] Destination: ..\data\raw\dementia_diaries\audio\Carol reports on the DEEP Gathering in Canterbury._641762535.wav



--- Download Interrupted by User (Ctrl+C) ---
Attempting to terminate yt-dlp...
yt-dlp process terminated.
Completed downloads are saved in the archive file. Run the script again to resume.


In [4]:
# --- Metadata Processing ---

print("\n--- Processing downloaded metadata ---")

json_files = glob.glob(str(output_base_dir / "audio/*.info.json"))
metadata_list = []
processed_ids = set() # To handle potential duplicates if script is run multiple times

if not json_files:
    print("No .info.json files found. Did the download run correctly and produce files?")
else:
    print(f"Found {len(json_files)} .info.json files. Parsing...")
    for json_file_path in json_files:
        try:
            with open(json_file_path, 'r', encoding='utf-8') as f:
                metadata = json.load(f)

            track_id = metadata.get("id")
            if not track_id or track_id in processed_ids:
                continue # Skip if no ID or already processed

            processed_ids.add(track_id)

            # Extract relevant information
            title = metadata.get("title", "Unknown Title")
            # The actual audio file path (yt-dlp saves it relative to where it was run, adjust if needed)
            # Assuming the WAV file is in the same directory with the same base name
            audio_filename_base = Path(json_file_path).stem
            wav_filename = f"{audio_filename_base}.{AUDIO_FORMAT}"
            # Construct path relative to the *script's* knowledge of output_base_dir
            relative_audio_path = Path("audio") / wav_filename

            # Attempt to parse diarist name (heuristic, might need manual cleaning)
            # Example: "Diarist Name - Title of Entry"
            diarist_name = title.split('-')[0].strip() if '-' in title else "Unknown Diarist"

            duration = metadata.get("duration") # Duration in seconds
            upload_date = metadata.get("upload_date") # Format YYYYMMDD
            description = metadata.get("description")

            metadata_list.append({
                "track_id": track_id,
                "diarist_name": diarist_name, # This is a guess based on title pattern
                "title": title,
                "label": 1, # All entries are Dementia
                "duration_seconds": duration,
                "upload_date": upload_date,
                "description": description,
                "relative_audio_path": str(relative_audio_path), # Path relative to output_base_dir
                "metadata_source": Path(json_file_path).name
            })

        except json.JSONDecodeError:
            print(f"Error decoding JSON: {json_file_path}")
        except Exception as e:
            print(f"Error processing file {json_file_path}: {e}")

    if metadata_list:
        df_metadata = pd.DataFrame(metadata_list)
        output_csv_path = output_base_dir / "dementia_diaries_metadata.csv"
        df_metadata.to_csv(output_csv_path, index=False)
        print(f"\nSuccessfully processed {len(df_metadata)} tracks.")
        print(f"Metadata saved to: {output_csv_path}")

        # Display first few rows
        print("\n--- Sample Metadata ---")
        print(df_metadata.head())
    else:
        print("No valid metadata could be extracted.")

print("\n--- Metadata processing finished ---")


--- Processing downloaded metadata ---
Found 1818 .info.json files. Parsing...

Successfully processed 1818 tracks.
Metadata saved to: ..\data\raw\dementia_diaries\dementia_diaries_metadata.csv

--- Sample Metadata ---
     track_id               diarist_name  \
0   664696877  'Keep going' says Stephen   
1  1454969251            Unknown Diarist   
2  1512632956            Unknown Diarist   
3   999247681            Unknown Diarist   
4  1284208405            Unknown Diarist   

                                               title  label  duration_seconds  \
0  'Keep going' says Stephen - fuelled by ice-cream!      1            65.965   
1  A beautiful day in the Brecon Beacons, but dis...      1            91.977   
2  A beautiful reading of Tarentella by Hilaire B...      1           105.143   
3  A beautiful reminder not to take for granted t...      1           228.702   
4  A big welcome from Wayne as Dementia Diaries r...      1            72.829   

  upload_date description   